<a href="https://colab.research.google.com/github/shiyao-zhao/Python-for-Maschinen-Learning/blob/main/%E6%89%8B%E5%86%99%E6%95%B0%E5%AD%97%E8%AF%86%E5%88%AB_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

线性回归-连续值预测-1.线性回归 2. 逻辑回归（二分类） 
离散值-分类


In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets,layers,optimizers,Sequential,metrics

In [7]:
def preprocess(x,y):
  x = tf.cast(x,dtype=tf.float32)/255.
  y = tf.cast(y,dtype=tf.int32)
  return x,y


(x,y),(x_test,y_test)=datasets.mnist.load_data()
print('datasets:',x.shape,y.shape)

datasets: (60000, 28, 28) (60000,)


In [9]:
batchsize = 128
#xs = tf.convert_to_tensor(xs,dtype=tf.float32)/255.
#ys = tf.convert_to_tensor(ys,dtype=tf.int32)
#y = tf.one_hot(y,depth=10)
db = tf.data.Dataset.from_tensor_slices((x,y))
db = db.map(preprocess).shuffle(10000).batch(batchsize)

db_test = tf.data.Dataset.from_tensor_slices((x_test,y_test))
db_test = db_test.map(preprocess).batch(batchsize)

db_iter = iter(db)
sample = next(db_iter)
print('batch:',sample[0].shape,sample[1].shape)

batch: (128, 28, 28) (128,)


In [12]:
model = keras.Sequential([
                          layers.Dense(512,activation='relu'),
                          layers.Dense(256,activation='relu'),
                          layers.Dense(128,activation='relu'),
                          layers.Dense(64,activation='relu'),
                          layers.Dense(10)
            
])

model.build(input_shape=[None,28*28])
model.summary()
optimizer = optimizers.Adam(lr=1e-3)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_8 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_9 (Dense)              (None, 10)                650       
Total params: 575,050
Trainable params: 575,050
Non-trainable params: 0
_________________________________________________________________


In [17]:
def main():
#optimizer =optimizers.SGD(learning_rate=0.001)

#def train_epoch(epoch):
  for epoch in range(5):
    
    for step,(x,y) in enumerate(db):
      x = tf.reshape(x,[-1,28*28])
      
      with tf.GradientTape() as tape:
        logits=model(x)
        y_onehot=tf.one_hot(y,depth=10)
        loss_mse= tf.reduce_mean(tf.losses.MSE(y_onehot,logits))
        loss_ce= tf.losses.categorical_crossentropy(y_onehot,logits,from_logits=True)
        loss_ce= tf.reduce_mean(loss_ce)

      grads=tape.gradient(loss_ce,model.trainable_variables)
      optimizer.apply_gradients(zip(grads,model.trainable_variables))

      if step % 100==0:
        print(epoch,step, 'loss', float(loss_ce),float(loss_mse))
    
    #test
    total_correct,total_num = 0.,0
    for x,y in db_test:
      x = tf.reshape(x,[-1,28*28])
      logits = model(x)
      prob = tf.nn.softmax(logits,axis=1)
      pred = tf.argmax(prob,axis=1)
      pred = tf.cast(pred, dtype=tf.int32)
      correct = tf.equal(pred,y)
      correct = tf.reduce_sum(tf.cast(correct,dtype=tf.int32))

      total_correct += int(correct)
      total_num += x.shape[0]

    acc = total_correct/total_num
    print(epoch,'test acc:',acc)

if __name__=='__main__':
  main()

0 0 loss 0.04727201163768768 19.65355682373047
0 100 loss 0.00931528676301241 26.714637756347656
0 200 loss 0.07901075482368469 24.83548927307129
0 300 loss 0.08683250844478607 25.064796447753906
0 400 loss 0.07721150666475296 27.182273864746094
0 test acc: 0.9745
1 0 loss 0.08123013377189636 28.154521942138672
1 100 loss 0.01036472711712122 32.96551513671875
1 200 loss 0.03449293226003647 29.08911895751953
1 300 loss 0.008725968189537525 30.90662384033203
1 400 loss 0.007078244816511869 32.86140060424805
1 test acc: 0.9747
2 0 loss 0.027657099068164825 32.189292907714844
2 100 loss 0.007016792893409729 34.42974853515625
2 200 loss 0.035310111939907074 37.70127487182617
2 300 loss 0.045026786625385284 37.193477630615234
2 400 loss 0.06983201950788498 30.281307220458984
2 test acc: 0.9788
3 0 loss 0.057517316192388535 35.15782928466797
3 100 loss 0.08328242599964142 45.74001693725586
3 200 loss 0.004992438945919275 41.43646240234375
3 300 loss 0.015180600807070732 40.531959533691406
3 4